In [ ]:
import face_detection
import imageio
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
import os

import warnings
warnings.filterwarnings("ignore")

In [ ]:
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

input_path = "input/"
output = "output_face_detection/"
output_roi = "output_roi/"
imgs_path = sorted(os.listdir(input_path))


for i, img in enumerate(tqdm(imgs_path)):
    
    img_path = img
    fig, ax = plt.subplots(1)
    img = imageio.imread(input_path+img)
    w, h, _ = img.shape

    img_detections = detector.detect(img)
    ax.imshow(img)
    
    for j, faces_detected in enumerate(img_detections):
        x1, y1, x2, y2, _ = faces_detected
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
        ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2))
        imageio.imwrite(f"{output_roi+img_path[:-4]}_output_{j}_roi.png",img[int(y1):int(y2), int(x1): int(x2)])
    
    plt.savefig(f"{output+img_path[:-4]}_output.png")
    plt.cla()
    plt.clf()
    plt.close()   


# skin classification

In [ ]:
from __future__ import print_function, division
import warnings
warnings.filterwarnings("ignore")
import os.path
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import dlib
import os
import argparse
from torchvision.models import resnet34

import matplotlib.pyplot as plt

In [ ]:
input_path = "output_roi/"
img_names = sorted(os.listdir(input_path))

output_path = "output_roi_skin_classification/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


bestW_7_path = "../skin_color_classification/fairface_official_weights/res34_fair_align_multi_7_20190809.pt"
model_fair_7 = torchvision.models.resnet34(pretrained=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
model_fair_7.load_state_dict(torch.load(bestW_7_path))
model_fair_7 = model_fair_7.to(device)
model_fair_7.eval()


bestW_4_path = "../skin_color_classification/fairface_official_weights/res34_fair_align_multi_4_20190809.pt"
model_fair_4 = resnet34(pretrained=True)
model_fair_4.fc = nn.Linear(model_fair_4.fc.in_features, 18)
model_fair_4.load_state_dict(torch.load(bestW_4_path))
model_fair_4.to(device)
model_fair_4.eval()

In [ ]:
trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


# img pth of face images
face_names = []


# list within a list. Each sublist contains scores for all races. 
# Take max for predicted race
race_scores_fair = []
gender_scores_fair = []
age_scores_fair = []
race_preds_fair = []
gender_preds_fair = []
age_preds_fair = []
race_scores_fair_4 = []
race_preds_fair_4 = []

In [ ]:
for index, img_name in enumerate(tqdm(img_names)):

    image = dlib.load_rgb_image(input_path+img_name)
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    image = trans(image)
    image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
    image = image.to(device)
    
    # fair 7 class
    outputs = model_fair_7(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)

    race_outputs = outputs[:7]
    gender_outputs = outputs[7:9]
    age_outputs = outputs[9:18]

    race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
    gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
    age_score = np.exp(age_outputs) / np.sum(np.exp(age_outputs))

    race_pred = np.argmax(race_score)
    gender_pred = np.argmax(gender_score)
    age_pred = np.argmax(age_score)

    race_scores_fair.append(race_score)
    gender_scores_fair.append(gender_score)
    age_scores_fair.append(age_score)

    race_preds_fair.append(race_pred)
    gender_preds_fair.append(gender_pred)
    age_preds_fair.append(age_pred)



    # fair 4 class
    outputs = model_fair_4(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)

    race_outputs = outputs[:4]
    race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
    race_pred = np.argmax(race_score)

    race_scores_fair_4.append(race_score)
    race_preds_fair_4.append(race_pred)    
    

    plt.savefig(f"{output_path+img_name[:-4]}_output.png")
    
    

In [ ]:
result = pd.DataFrame([face_names,
                    race_preds_fair,
                    race_preds_fair_4,
                    gender_preds_fair,
                    age_preds_fair,
                    race_scores_fair, race_scores_fair_4,
                    gender_scores_fair,
                    age_scores_fair, ]).T


result.columns = ['face_name_align',
                'race_preds_fair',
                'race_preds_fair_4',
                'gender_preds_fair',
                'age_preds_fair',
                'race_scores_fair',
                'race_scores_fair_4',
                'gender_scores_fair',
                'age_scores_fair']


result.loc[result['race_preds_fair'] == 0, 'race'] = 'White'
result.loc[result['race_preds_fair'] == 1, 'race'] = 'Black'
result.loc[result['race_preds_fair'] == 2, 'race'] = 'Latino_Hispanic'
result.loc[result['race_preds_fair'] == 3, 'race'] = 'East Asian'
result.loc[result['race_preds_fair'] == 4, 'race'] = 'Southeast Asian'
result.loc[result['race_preds_fair'] == 5, 'race'] = 'Indian'
result.loc[result['race_preds_fair'] == 6, 'race'] = 'Middle Eastern'

# race fair 4
result.loc[result['race_preds_fair_4'] == 0, 'race4'] = 'White'
result.loc[result['race_preds_fair_4'] == 1, 'race4'] = 'Black'
result.loc[result['race_preds_fair_4'] == 2, 'race4'] = 'Asian'
result.loc[result['race_preds_fair_4'] == 3, 'race4'] = 'Indian'

# gender
result.loc[result['gender_preds_fair'] == 0, 'gender'] = 'Male'
result.loc[result['gender_preds_fair'] == 1, 'gender'] = 'Female'

# age
result.loc[result['age_preds_fair'] == 0, 'age'] = '0-2'
result.loc[result['age_preds_fair'] == 1, 'age'] = '3-9'
result.loc[result['age_preds_fair'] == 2, 'age'] = '10-19'
result.loc[result['age_preds_fair'] == 3, 'age'] = '20-29'
result.loc[result['age_preds_fair'] == 4, 'age'] = '30-39'
result.loc[result['age_preds_fair'] == 5, 'age'] = '40-49'
result.loc[result['age_preds_fair'] == 6, 'age'] = '50-59'
result.loc[result['age_preds_fair'] == 7, 'age'] = '60-69'
result.loc[result['age_preds_fair'] == 8, 'age'] = '70+'


save_predictions = "out.csv"
result[['face_name_align',
    'race', 'race4',
    'gender', 'age',
    'race_scores_fair', 'race_scores_fair_4',
    'gender_scores_fair', 'age_scores_fair']].to_csv(save_predictions, index=False)